In [1]:
import numpy as np
import pandas as pd
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import mdtraj
import sys
import os

try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../..')
from openabc.forcefields.parsers import MpipiProteinParser, MpipiRNAParser
from openabc.forcefields.mpipi_model import MpipiModel
import openabc.utils.helper_functions as helper_functions
from openabc.utils.insert import insert_molecules


Here we show how to use Mpipi model to set up protein-RNA simulations. We use a mixture of polyR, polyK, and polyU as an example. First we build individual CG chains with given sequence, then we insert chains into a simulation box. Finally we run the MD simulation. 

In [2]:
# build residue level CG atom chains
polyR_atoms = helper_functions.build_straight_CA_chain('RRRRRRRRRR')
polyR_atoms.loc[:, 'chainID'] = 'A'
helper_functions.write_pdb(polyR_atoms, 'polyR_CA.pdb')
polyK_atoms = helper_functions.build_straight_CA_chain('KKKKKKKKKK')
polyK_atoms.loc[:, 'chainID'] = 'B'
helper_functions.write_pdb(polyK_atoms, 'polyK_CA.pdb')
polyU_atoms = helper_functions.build_straight_chain(n_atoms=10, chainID='C', r0=0.5) # for RNA, r0 is 0.5 nm
polyU_atoms.loc[:, 'name'] = 'RN' # set CG nucleotide atom name as RN
polyU_atoms.loc[:, 'resname'] = 'U' # residue name is U
helper_functions.write_pdb(polyU_atoms, 'polyU_CG.pdb')

# parse individual protein and RNA
polyR = MpipiProteinParser('polyR_CA.pdb')
polyK = MpipiProteinParser('polyK_CA.pdb')
polyU = MpipiRNAParser('polyU_CG.pdb')

Parse molecule with default settings.
Parse molecule with default settings.
Parse molecule with default settings.


In [3]:
# insert chains into a box
# for simplicity, just insert 2 copies of each sequence
insert_molecules('polyR_CA.pdb', 'tmp1.pdb', n_mol=2, box=[20, 20, 20])
insert_molecules('polyK_CA.pdb', 'tmp2.pdb', n_mol=2, existing_pdb='tmp1.pdb', box=[20, 20, 20])
insert_molecules('polyU_CG.pdb', 'start.pdb', n_mol=2, existing_pdb='tmp2.pdb', box=[20, 20, 20])

Successfully inserted 2 molecules.
Successfully inserted 2 molecules.
Successfully inserted 2 molecules.


In [4]:
# visualize start.pdb
start_pdb = mdtraj.load_pdb('start.pdb')
view = nglview.show_mdtraj(start_pdb)
view

NGLWidget()

In [5]:
# set up simulation
top = app.PDBFile('start.pdb').getTopology()
protein_rna = MpipiModel()
for i in range(2):
    protein_rna.append_mol(polyR)
for i in range(2):
    protein_rna.append_mol(polyK)
for i in range(2):
    protein_rna.append_mol(polyU)
protein_rna.create_system(top, box_a=20, box_b=20, box_c=20)
protein_rna.add_protein_bonds(force_group=1)
protein_rna.add_rna_bonds(force_group=2)
protein_rna.add_contacts(force_group=3)
protein_rna.add_dh_elec(ldby=(1/1.26)*unit.nanometer, force_group=4)
temperature = 300*unit.kelvin
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('start.pdb').getPositions()
protein_rna.set_simulation(integrator, platform_name='CPU', init_coord=init_coord)
protein_rna.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output.dcd'
protein_rna.add_reporters(output_interval, output_dcd)
protein_rna.simulation.context.setVelocitiesToTemperature(temperature)
protein_rna.simulation.step(5000)

Add protein bonds.
Add RNA bonds.
Add nonbonded contacts.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 0.7936507936507936 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,104.82590655320087,226.29463480632816,331.12054135952906,307.53659504735293,0
200,2.0000000000000013,102.83935551709224,187.47118203139945,290.3105375484917,254.77514763344632,211
300,2.99999999999998,84.80793102350162,223.22888850752935,308.03681953103097,303.37021620757827,206
400,3.9999999999999587,120.44055210880184,225.08098616262842,345.52153827143025,305.88723481489893,208
500,4.999999999999938,83.47948299019694,230.77113748343567,314.2506204736326,313.62020543527143,208
600,5.9999999999999165,112.33935966990101,212.40935963526883,324.74871930516986,288.66637193730094,210
700,6.999999999999895,92.38802375657855,211.0146042696783,303

In [6]:
# view trajectory
traj = mdtraj.load_dcd('output.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=49)